In [1]:
# import os 
# os.environ['KERAS_BACKEND'] = 'tensorflow'
import sys 
sys.path.append('../')

import pandas as pd
from sklearn.model_selection import train_test_split

# import keras
# import keras_nlp
# from keras_nlp.models import BertTextClassifier

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import(
    BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
)

/opt/miniconda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fake_df = pd.read_csv('../data/Fake.csv')
real_df = pd.read_csv('../data/True.csv')

print(fake_df.head())
print(fake_df.subject.value_counts())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
subject
News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east 

In [3]:
# Step 1: Prepare the data
# Label the data: 1 for true, 0 for fake
fake_df['label'] = 0
real_df['label'] = 1
data_df = pd.concat([fake_df, real_df], ignore_index=True)
data_df['content'] = data_df['title'] + ' ' + data_df['text']
train_df, test_df = train_test_split(data_df[['content', 'label']], test_size=0.2, random_state=42)

# Step 2: Define a custom Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return { 'input_ids': encoding['input_ids'].squeeze(), 'attention_mask': encoding['attention_mask'].squeeze(), 'labels': torch.tensor(label) }

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create train and test datasets
train_dataset = TextDataset(train_df['content'].tolist(), train_df['label'].tolist(), tokenizer)
test_dataset = TextDataset(test_df['content'].tolist(), test_df['label'].tolist(), tokenizer)

# Step 3: Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Step 4: Define TrainingArguments and Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Step 5: Train the model
trainer.train()

# Step 6: Evaluate the model
eval_results = trainer.evaluate()
print("Test Accuracy:", eval_results['eval_accuracy'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda/envs/py39/lib/python3.9/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [33]:
fake_df['label'] = 0
real_df['label'] = 1

df = pd.concat([fake_df, real_df], axis=0)
df['text'] = df['title'] + ' ' + df['text']

excess, train = train_test_split(df, test_size=0.2, stratify=df['subject'])
train, test = train_test_split(train, test_size=0.2, stratify=train['subject'])

In [55]:
classification_model = keras_nlp.models.BertTextClassifier.from_preset(
    "bert_base_en_uncased", 
    num_classes=1, 
    # preprocessor = None
    # preprocessor = keras_nlp.models.BertPreprocessor.from_preset( 
    #     "bert_base_en_uncased", 
    # )
)

In [56]:
#classification_model.backbone.trainable = False
classification_model.summary()

Preprocessor: "bert_text_classifier_preprocessor_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ bert_tokenizer (BertTokenizer)                                │                       Vocab size: 30,522 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "bert_text_classifier_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ segment_ids (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bert_backbone (BertBackbone)  │ [(None, 768), (None,      │     109,482,240 │ padding_mask[0][0],        │
│                               │ None, 768)]               │                 │ segment_ids[0][0],         │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ classifier_dropout (Dropout)  │ (None, 768)               │               0 │ bert_backbone[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 1)                 │             769 │ classifier_dropout[0][0]   │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
classification_model.compile(
    loss=keras.losses.BinaryCrossentropy(), 
    #optimizer=keras.optimizers.Adam(), 
    optimizer=keras.optimizers.RMSprop(clipnorm=1.0), 
    metrics=[keras.metrics.BinaryAccuracy, keras.metrics.AUC]
)

history = classification_model.fit(
    x=train['text'], y=train['label'], batch_size=8, 
    validation_split=0.1, epochs=10  
)

Epoch 1/10


In [29]:
classification_model.evaluate(x=test['text'], y=test['label'], batch_size=10)

898/898 ━━━━━━━━━━━━━━━━━━━━ 55s 59ms/step - loss: nan - sparse_categorical_accuracy: 0.4631


[nan, 0.47750556468963623]